In [1]:
import numpy as np
import open3d as o3d
import copy
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.spatial import cKDTree
from joblib import Parallel, delayed

from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.IFSelect import IFSelect_RetDone
from OCC.Core.TopAbs import TopAbs_WIRE, TopAbs_EDGE, TopAbs_VERTEX
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods
from OCC.Core.BRep import BRep_Tool
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve
from OCC.Core.GeomAbs import GeomAbs_Circle, GeomAbs_Ellipse, GeomAbs_Line, GeomAbs_BSplineCurve, GeomAbs_Hyperbola, GeomAbs_Parabola
from OCC.Core.TopAbs import TopAbs_WIRE, TopAbs_EDGE, TopAbs_VERTEX
from OCC.Display.SimpleGui import init_display
from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB
from OCC.Core.gp import gp_Pnt
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeVertex
from PyQt5 import QtWidgets
from OCC.Core.GCPnts import GCPnts_UniformDeflection
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Read processed point cloud data

point_dictionary_path = '/home/chris/Code/PointClouds/data/hull_pcd/morten_plate/mapped_hull_points_dictionary.pkl'

hull_centers_dictionary = {}

with open(point_dictionary_path, 'rb') as f:
    hull_points_dictionary = pickle.load(f)

hull_pcd = o3d.geometry.PointCloud()

for label, points in hull_points_dictionary.items():
    hull_pcd.points.extend(o3d.utility.Vector3dVector(points))
    center = np.mean(points, axis=0)
    hull_centers_dictionary[label] = center
    
hull_pcd.paint_uniform_color([0.1, 1, 0.1])

hull_centers_pcd = o3d.geometry.PointCloud()
hull_centers_points = list(hull_centers_dictionary.values())
hull_centers_pcd.points = o3d.utility.Vector3dVector(np.array(hull_centers_points))
hull_centers_pcd.paint_uniform_color([1, 0, 0])  # Color centers red

print("Point cloud data has been loaded successfully.")

Point cloud data has been loaded successfully.


In [3]:
# Read cad model data (STEP file)

step_file_path = '/home/chris/Code/PointClouds/data/other_files/MortenPartSTEPVersion.STEP'

def load_step_file(file_path):
    step_reader = STEPControl_Reader()
    status = step_reader.ReadFile(file_path)
    if status == IFSelect_RetDone:
        step_reader.TransferRoots()
        shape = step_reader.OneShape()
        return shape
    else:
        raise Exception("Error: Cannot read STEP file.")

shape = load_step_file(step_file_path)

print('Step file has been loaded successfully.')

Step file has been loaded successfully.


In [4]:
# Extract edges at the top Z-coordinate
def extract_top_edges(shape, deflection=0.1):
    edges = []
    max_z = -float('inf')
    edge_z_coordinates = []

    # First pass to collect the highest Z-coordinate of each edge
    exp_edge = TopExp_Explorer(shape, TopAbs_EDGE)
    while exp_edge.More():
        edge = topods.Edge(exp_edge.Current())
        edge_max_z = -float('inf')
        exp_vertex = TopExp_Explorer(edge, TopAbs_VERTEX)
        vertices = []
        while exp_vertex.More():
            vertex = topods.Vertex(exp_vertex.Current())
            point = BRep_Tool.Pnt(vertex)
            vertices.append(point)
            if point.Z() > edge_max_z:
                edge_max_z = point.Z()
            exp_vertex.Next()
        edge_z_coordinates.append((edge, edge_max_z, vertices))
        if edge_max_z > max_z:
            max_z = edge_max_z
        exp_edge.Next()

    #print(f"Total number of edges: {len(edge_z_coordinates)}")
    #print(f"Maximum Z-coordinate found: {max_z}")

    # Collect edges on the top plane
    top_edges = []
    point_cloud = []
    for edge, z, vertices in edge_z_coordinates:
        if all(abs(vertex.Z() - max_z) < 1e-3 for vertex in vertices):  # Ensure all vertices are at the top surface
            top_edges.append(edge)
            #print(f"Top edge added with highest vertex Z: {z}")

            # Sample points along the edge
            curve = BRepAdaptor_Curve(edge)
            u_min, u_max = curve.FirstParameter(), curve.LastParameter()

            # Calculate the length of the edge
            linear_props = GProp_GProps()
            brepgprop.LinearProperties(edge, linear_props)
            length = linear_props.Mass()

            # Calculate the number of samples based on the length and the desired point density
            num_samples = int(length / deflection)

            for i in range(num_samples):
                u = u_min + i * (u_max - u_min) / (num_samples - 1)
                pnt = curve.Value(u)
                point_cloud.append((pnt.X(), pnt.Y(), pnt.Z()))  # Store the coordinates as a tuple

    print(f"Total number of top view edges: {len(top_edges)}")
    print(f"Total number of points in the cad point cloud: {len(point_cloud)}")
    return top_edges, point_cloud

# Visualize the top edges
def visualize_top_edges(top_edges):
    display, start_display, add_menu, add_function_to_menu = init_display()

    # Assign the color red to each top edge
    red_color = Quantity_Color(1.0, 0.0, 0.0, Quantity_TOC_RGB)
    
    # Display each top edge with the red color
    for edge in top_edges:
        display.DisplayShape(edge, update=True, color=red_color)

    start_display()


# Main execution
print('Generating top view edges and sampling to create the associated point cloud...')
top_edges, cad_points = extract_top_edges(shape)
# if top_edges:
#     visualize_top_edges(top_edges)
# else:
#     print("Top edges not found.")

cad_pcd = o3d.geometry.PointCloud()
cad_pcd.points = o3d.utility.Vector3dVector(np.asarray(cad_points))
#o3d.visualization.draw_geometries([cad_pcd])

print('Top view cad point cloud created.')

Generating top view edges and sampling to create the associated point cloud...
Total number of top view edges: 586
Total number of points in the cad point cloud: 81774
Top view cad point cloud created.


In [5]:
# Apply clustering to the cad_pcd

eps = 0.50
min_points = 10

cad_shapes_labels = np.array(cad_pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))

max_label = cad_shapes_labels.max()
print(f"Cad point cloud has {max_label + 1} clusters")
base_cmap = plt.get_cmap("tab20")
color_cycle = [base_cmap(i % 20) for i in range(max_label + 1)]
colors = np.array(color_cycle)[cad_shapes_labels]
colors[cad_shapes_labels<0] = 0
cad_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

cad_points_dictionary = {}
cad_centers_dictionary = {}

# Extract the points and calculate the centers for each cluster
for label in np.unique(cad_shapes_labels):
    if label != -1:  # Ignore noise
        cluster_points = np.asarray(cad_pcd.points)[cad_shapes_labels == label]
        cad_points_dictionary[label] = cluster_points
        cad_centers_dictionary[label] = np.mean(cluster_points, axis=0)

cad_centers_pcd = o3d.geometry.PointCloud()
cad_centers_points = list(cad_centers_dictionary.values())
cad_centers_pcd.points = o3d.utility.Vector3dVector(np.array(cad_centers_points))
cad_centers_pcd.paint_uniform_color([0, 1, 0])  # Color centers green

#o3d.visualization.draw_geometries([cad_pcd])

Precompute neighbors.[========================================] 100%
ClusteringCad point cloud has 79 clusters          ] 15%
Clustering[========================================] 100%


PointCloud with 79 points.

In [6]:
# Center based alignment: Align point clouds through ICP based on the lowest RMSE (only use centers of shapes)

cad_centers_pcd_center = np.mean(np.asarray(cad_centers_pcd.points), axis=0)
hull_centers_pcd_center = np.mean(np.asarray(hull_centers_pcd.points), axis=0)

# Compute the translation vector
translation = cad_centers_pcd_center - hull_centers_pcd_center

# Translate the hull_pcd to align the centers
hull_centers_pcd.points = o3d.utility.Vector3dVector(np.asarray(hull_centers_pcd.points) + translation)

#o3d.visualization.draw_geometries([cad_centers_pcd, hull_centers_pcd])

In [10]:
visualization = o3d.visualization.VisualizerWithKeyCallback()
visualization.create_window()
visualization.get_render_option().background_color = np.asarray([0.95, 0.95, 0.95])

class RansacState:
    def __init__(self, pcd_A, pcd_B):
        print('Initializing Parameters:')
        print(30*'-')

        self.pcd_A = pcd_A
        self.pcd_B = pcd_B

        self.cloud_A_points = np.asarray(pcd_A.points)[:, :2]
        self.cloud_B_points = np.asarray(pcd_B.points)[:, :2]

        self.pcd_A_colors = np.tile((0.1,0.9,0.1), (self.cloud_A_points.shape[0], 1))
        self.pcd_B_colors = np.tile((0.1,0.1,0.9), (self.cloud_B_points.shape[0], 1))

        print(f'cloud_A shape: {self.cloud_A_points.shape}')
        print(f'cloud_B shape: {self.cloud_B_points.shape}')

        self.cloud_A_indices = np.arange(self.cloud_A_points.shape[0])
        self.cloud_B_indices = np.arange(self.cloud_B_points.shape[0])
        
        self.changing_angle = False
        self.picked_pair = None
        self.added_geometries = False
        self.current_angle_idx = 0
        self.it = -1

        self.angles = np.arange(-np.pi/2, np.pi/2, np.pi/720)
        print(30*'-')

        combinations = np.array(np.meshgrid(self.cloud_A_indices, self.cloud_B_indices)).T.reshape(-1, 2)
        self.shuffled_combinations = np.random.permutation(combinations)
        print(f'shuffled_combinations shape: {self.shuffled_combinations.shape}')
        print('printing the first five elements from shuffled_combinations:')
        for i in range(5):
            print(self.shuffled_combinations[i])
        print(30*'-')

        self.translations = self.cloud_B_points[self.shuffled_combinations[:, 1]] - self.cloud_A_points[self.shuffled_combinations[:, 0]]
        print(f'translations shape {self.translations.shape}')
        print('printing the first five elements of translations:')
        for i in range(5):
            print(self.translations[i])
        print(30*'-')

        self.translated_clouds = self.cloud_A_points[np.newaxis, :, :] + self.translations[:, np.newaxis, :]
        self.centering_vectors = self.cloud_B_points[self.shuffled_combinations[:, 1]]
        self.centered_clouds = self.translated_clouds - self.centering_vectors[:, np.newaxis, :]

        print(f'translated_clouds shape: {self.translated_clouds.shape}')
        print(f'centered_clouds shape: {self.centered_clouds.shape}')
        print(30*'-')
        return
    
    def get_rotation_matrix(self,angle):
        return np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])

    def apply_single_rotation(self, centered_cloud, angle):
        rotation_matrix = self.get_rotation_matrix(angle)
        rotated_cloud = np.dot(centered_cloud, rotation_matrix.T)
        return rotated_cloud

    def calculate_distances_optimized(self, rotated_cloud, cloud_B):
        tree = cKDTree(cloud_B)
        distances, _ = tree.query(rotated_cloud)
        return distances
    
    def pair_points_forward_callback(self, vis):
        if self.added_geometries is False:
            self.pcd_A.colors = o3d.utility.Vector3dVector(self.pcd_A_colors)
            self.pcd_B.colors = o3d.utility.Vector3dVector(self.pcd_B_colors)
            visualization.add_geometry(self.pcd_A)
            visualization.add_geometry(self.pcd_B)
            self.added_geometries = True
            return
        else:
            self.it += 1
            new_pcd_A_points = self.translated_clouds[self.it]
            new_pcd_A_points_3d = np.hstack((new_pcd_A_points, np.full((new_pcd_A_points.shape[0], 1), 2)))
            self.pcd_A.points = o3d.utility.Vector3dVector(new_pcd_A_points_3d)
            self.picked_pair = self.shuffled_combinations[self.it]
            self.pcd_A_colors[self.picked_pair[0]] = (0.9, 0.1, 0.1)
            self.pcd_B_colors[self.picked_pair[1]] = (0.9, 0.1, 0.1)
            self.pcd_A.colors = o3d.utility.Vector3dVector(self.pcd_A_colors)
            self.pcd_B.colors = o3d.utility.Vector3dVector(self.pcd_B_colors)
            vis.update_geometry(self.pcd_A)
            vis.update_geometry(self.pcd_B)
            self.pcd_A_colors[self.picked_pair[0]] = (0.1, 0.9, 0.1)
            self.pcd_B_colors[self.picked_pair[1]] = (0.1, 0.1, 0.9)
            self.current_angle_idx = 0
            self.changing_angle = False
        return
    
    def pair_points_backward_callback(self, vis):
        if self.added_geometries and self.it >= 1:
            self.it -= 1
            new_pcd_A_points = self.translated_clouds[self.it]
            new_pcd_A_points_3d = np.hstack((new_pcd_A_points, np.full((new_pcd_A_points.shape[0], 1), 2)))
            self.pcd_A.points = o3d.utility.Vector3dVector(new_pcd_A_points_3d)
            self.picked_pair = self.shuffled_combinations[self.it]
            self.pcd_A_colors[self.picked_pair[0]] = (0.9, 0.1, 0.1)
            self.pcd_B_colors[self.picked_pair[1]] = (0.9, 0.1, 0.1)
            self.pcd_A.colors = o3d.utility.Vector3dVector(self.pcd_A_colors)
            self.pcd_B.colors = o3d.utility.Vector3dVector(self.pcd_B_colors)
            vis.update_geometry(self.pcd_A)
            vis.update_geometry(self.pcd_B)
            self.pcd_A_colors[self.picked_pair[0]] = (0.1, 0.9, 0.1)
            self.pcd_B_colors[self.picked_pair[1]] = (0.1, 0.1, 0.9)
            self.current_angle_idx = 0
            self.changing_angle = False
        return
    
    def rotate_cloud_forward_callback(self, vis):
        if self.changing_angle == False:
            angle = self.angles[self.current_angle_idx]
            centered_cloud = self.centered_clouds[self.it]
            centering_vector = self.centering_vectors[self.it]

            rotated_cloud = self.apply_single_rotation(centered_cloud, angle)
            uncentered_rotated_cloud = rotated_cloud + centering_vector

            distances = self.calculate_distances_optimized(uncentered_rotated_cloud, self.cloud_B_points)
            avg_distance = np.mean(distances)
            print(f"Angle: {angle:.2f} radians, Average Distance: {avg_distance:.4f}", flush=True)
            uncentered_rotated_cloud_3d = np.hstack((uncentered_rotated_cloud, np.full((uncentered_rotated_cloud.shape[0], 1), 2)))
            self.pcd_A.points = o3d.utility.Vector3dVector(uncentered_rotated_cloud_3d)
            vis.update_geometry(self.pcd_A)
            self.changing_angle = True
            return
        
        if self.it != 0 and (self.current_angle_idx < self.angles.shape[0]):
            self.current_angle_idx += 1
            angle = self.angles[self.current_angle_idx]
            centered_cloud = self.centered_clouds[self.it]
            centering_vector = self.centering_vectors[self.it]

            rotated_cloud = self.apply_single_rotation(centered_cloud, angle)
            uncentered_rotated_cloud = rotated_cloud + centering_vector

            distances = self.calculate_distances_optimized(uncentered_rotated_cloud, self.cloud_B_points)
            avg_distance = np.mean(distances)
            print(f"Angle: {angle:.2f} radians, Average Distance: {avg_distance:.4f}", flush=True)
            uncentered_rotated_cloud_3d = np.hstack((uncentered_rotated_cloud, np.full((uncentered_rotated_cloud.shape[0], 1), 2)))
            self.pcd_A.points = o3d.utility.Vector3dVector(uncentered_rotated_cloud_3d)
            vis.update_geometry(self.pcd_A)
        return
    
    def rotate_cloud_backward_callback(self, vis):
        if self.it != 0 and (self.current_angle_idx > 0) and self.changing_angle:
            self.current_angle_idx -= 1
            angle = self.angles[self.current_angle_idx]
            centered_cloud = self.centered_clouds[self.it]
            centering_vector = self.centering_vectors[self.it]

            rotated_cloud = self.apply_single_rotation(centered_cloud, angle)
            uncentered_rotated_cloud = rotated_cloud + centering_vector

            distances = self.calculate_distances_optimized(uncentered_rotated_cloud, self.cloud_B_points)
            avg_distance = np.mean(distances)
            print(f"Angle: {angle:.2f} radians, Average Distance: {avg_distance:.4f}", flush=True)
            uncentered_rotated_cloud_3d = np.hstack((uncentered_rotated_cloud, np.full((uncentered_rotated_cloud.shape[0], 1), 2)))
            self.pcd_A.points = o3d.utility.Vector3dVector(uncentered_rotated_cloud_3d)
            vis.update_geometry(self.pcd_A)
        return

state = RansacState(cad_centers_pcd, hull_centers_pcd)
visualization.register_key_callback(262, state.pair_points_forward_callback)
visualization.register_key_callback(263, state.pair_points_backward_callback)
visualization.register_key_callback(265, state.rotate_cloud_forward_callback)
visualization.register_key_callback(264, state.rotate_cloud_backward_callback)
visualization.run()
visualization.destroy_window()

Initializing Parameters:
------------------------------
cloud_A shape: (79, 2)
cloud_B shape: (75, 2)
------------------------------
shuffled_combinations shape: (5925, 2)
printing the first five elements from shuffled_combinations:
[14  5]
[56 24]
[ 3 72]
[29 32]
[ 5 48]
------------------------------
translations shape (5925, 2)
printing the first five elements of translations:
[-81.31816911  96.16226572]
[-84.28340281 128.35110426]
[118.76551141  -1.07195402]
[ 3.49600014 50.47015615]
[57.38067023 91.25091388]
------------------------------
translated_clouds shape: (5925, 79, 2)
centered_clouds shape: (5925, 79, 2)
------------------------------
Angle: -1.57 radians, Average Distance: 30.9198
Angle: -1.57 radians, Average Distance: 30.8476
Angle: -1.56 radians, Average Distance: 30.7708
Angle: -1.56 radians, Average Distance: 30.6897
Angle: -1.55 radians, Average Distance: 30.6095
Angle: -1.55 radians, Average Distance: 30.5310
Angle: -1.54 radians, Average Distance: 30.4524
Angle: 